# Installations and imports

In [ ]:
!pip install accelerate
!pip install bitsandbytes

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

In [ ]:
!pip install datasets
from datasets import load_dataset

In [ ]:
import re
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Запросы имеют следующий вид в зависимости от параметра shots:

# Choosing model and dataset

In [ ]:
dataset = load_dataset("mbpp")
dataset = dataset['test']
dataset

Generating train split:   0%|          | 0/374 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/90 [00:00<?, ? examples/s]

Generating prompt split:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
    num_rows: 500
})

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = "codellama/CodeLlama-7b-Instruct-hf"
# #model = "deepseek-ai/deepseek-coder-1.3b-instruct"
# tokenizer = AutoTokenizer.from_pretrained(model)
# model = AutoModelForCausalLM.from_pretrained(model, torch_dtype=torch.float16, device_map='auto', load_in_8bit=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = "deepseek-ai/deepseek-coder-6.7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model, trust_remote_code=True, torch_dtype=torch.bfloat16, device_map='auto', load_in_8bit=True)

tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

# Generation and data preparation functions

In [ ]:
def make_prompt_list(dataset, shots, num_tests):
    prompt_list = []
    pattern = r'(?<=assert\s)\w+\s*\('
    for i in range(num_tests):
        s = dataset['test_list'][i][0]
        func_name = re.search(pattern, s)
        func_name = func_name.group().strip(' (') if func_name else ''
        prompt = dataset['text'][i] + ' The function should have the following name: ' + func_name + '.\n'
        if shots > 0:
            prompt += 'The code should also pass these tests: '
            for j in range(shots):
                prompt +=  dataset['test_list'][i][j] + ', '
            prompt = prompt[:len(prompt) - 2]
        prompt_list.append(prompt)
    return prompt_list

def generate(model, dataset, shots=0, num_tests=len(dataset), do_sample=False, top_p=-1.0, top_k=0, temperature=1.0):
    prompt_list = make_prompt_list(dataset, shots, num_tests)
    model.eval()
    responses = []

    for prompt in prompt_list:
        messages=[
            { 'role': 'user', 'content': prompt}
        ]
        inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(input_ids=inputs, max_new_tokens=200, num_return_sequences=1, do_sample=do_sample, top_p=top_p, top_k=top_k, temperature=temperature, eos_token_id=tokenizer.eos_token_id)
            response = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

        responses.append(response)
    return prompt_list, responses

In [ ]:
prompt_list = make_prompt_list(dataset, 0, 5)
print(prompt_list[0])
prompt_list = make_prompt_list(dataset, 3, 5)
print(prompt_list[0])

Write a python function to remove first and last occurrence of a given character from the string. The function should have the following name: remove_Occ.

Write a python function to remove first and last occurrence of a given character from the string. The function should have the following name: remove_Occ.
The code should also pass these tests: assert remove_Occ("hello","l") == "heo", assert remove_Occ("abcda","a") == "bcd", assert remove_Occ("PHP","P") == "H"


# Testing

In [ ]:
def extract_code(text):
    code = []
    inside_function = False

    for line in text.split('\n'):
        # if line.startswith('import') or line.startswith('from') or line.startswith('def') and inside_function == False:
        if line.startswith('```python'):
            inside_function = True
        elif inside_function:
            if line == '```' or line.startswith('assert') or line.startswith('# Test'):
                return '\n'.join(code)
            else:
                code.append(line)
    return '\n'.join(code)


In [ ]:
def test(num_tests, dataset, responses):
  score = 0
  codes = []
  tests = []
  results = []
  for i in range(num_tests):
    code = extract_code(responses[i])
    codes.append(code)
    test = dataset['test_list'][i][0] + '\n' + dataset['test_list'][i][1] + '\n' + dataset['test_list'][i][2]
    tests.append(test)
    code = code + '\n' + test if code else test
    code = code.strip()
    flag = True
    try:
      exec(code)
    except (AssertionError, TypeError, IndentationError, NameError, SyntaxError, AttributeError, EOFError, ImportError, IndexError, KeyError, ValueError, ZeroDivisionError):
      flag = False
      pass
    else:
      score += 1
    results.append('Ok' if flag else 'Error')
  return codes, tests, results

In [ ]:
def save_results(df, parameters, prompt_list, responses, codes, tests, results):
    bias = len(df)
    df.loc[bias, 'parameters'] = parameters
    for i in range(len(prompt_list)):
        df.loc[bias + i, 'prompt'] = prompt_list[i]
        df.loc[bias + i, 'response'] = responses[i]
        df.loc[bias + i, 'code'] = codes[i]
        df.loc[bias + i, 'tests'] = tests[i]
        df.loc[bias + i, 'result'] = results[i]

In [ ]:
def predict(df, model, dataset, shots=0, num_tests=100, do_sample=False, top_p=-1.0, top_k=0, temperature=1.0):
    prompt_list, responses = generate(model=model, dataset=dataset, shots=shots, num_tests=num_tests, do_sample=False, top_p=top_p, top_k=top_k, temperature=temperature)
    codes, tests, results = test(dataset=dataset, num_tests=num_tests, responses=responses)
    parameters = str({'shots': shots, 'do_sample': do_sample, 'top_p': top_p, 'top_k': top_k, 'temperature': temperature})
    save_results(df, parameters, prompt_list, responses, codes, tests, results)

# Getting results

In [ ]:
NUM_TESTS = 10

In [ ]:
df = pd.DataFrame(columns = ['parameters', 'prompt', 'response', 'code', 'tests', 'result'])

for shots in tqdm([0, 1, 2]):
    predict(df=df, model=model, dataset=dataset, shots=shots, num_tests=NUM_TESTS, do_sample=False)

for shots in tqdm([0, 1, 2]):
    for top_p in tqdm([0.9, 0.92, 0.95]):
        for top_k in tqdm([30, 50]):
            for temperature in tqdm([0.01, 0.05, 0.1, 0.25]):
                predict(df, model=model, dataset=dataset, shots=shots, num_tests=NUM_TESTS, do_sample=True, top_k=top_k, top_p=top_p, temperature=temperature)

# Saving results

In [47]:
df.to_csv('updated_data.csv', index=False)